# Proyecto 2
## RSNA 2022 Cervical Spine Fracture Detection
**Universidad del Valle de Guatemala**\
**Facultad de Ingeniería**\
**Departamento de Ciencias de la Computación**\
**Data Science**

---

## Integrantes
- Gustavo Gonzalez
- Pablo Orellana
- Diego Leiva
- Maria Ramirez

## Librerias

NOTA: Importante tener el Token de la API en el folder `C:\Users\<Windows-username>\.kaggle\kaggle.json`

Para una guia mas detallada o si tiene otro sistema dirigase a: https://www.kaggle.com/docs/api#getting-started-installation-&-authentication 

Asimismo puede visitar la documentacion oficial de la API: https://github.com/Kaggle/kaggle-api/tree/main/docs#api-credentials

Si no utilizara la API de Kaggle para descargar el conjunto de datos comente las lineas para importar la libreria de kaggle.

In [1]:
# Kaggle API
import kaggle
from kaggle.rest import ApiException

# Operaciones de archivos
import os
import pickle
import zipfile
import shutil

# Operaciones de datos
import pandas as pd
import numpy as np
from collections import defaultdict

# Operaciones de imágenes
from PIL import Image

# Visualización
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

# Utilidades
import random
import time
import uuid

# Pytorch y Torchvision
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import transforms

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# Warnings
# import warnings
# warnings.filterwarnings('ignore')

### Configuración de variable de entorno para CUDA
De no tener un equipo compatible con CUDA, o si ya tiene configurado el PATH de CUDA, comente la siguiente línea.
Para mas detalle puede visitar las guias de CUDA Toolkit en el siguiente enlace: https://docs.nvidia.com/cuda/archive/12.4.0/

In [2]:
# Ruta para CUDA 12.4
# os.environ['CUDA_PATH'] = r"C:/Program Files/NVIDIA GPU Computing Toolkit//CUDA//v12.4"  

## Carga de Dataset

In [3]:
# Definir competencia y directorio de conjunto de datos
competition_name = 'rsna-2022-cervical-spine-fracture-detection'
target_path = 'data'

# Bandera para habilitar o deshabilitar la descarga de datos
download_data = False

### Descargar el archivo con Kaggle

Puede omitir esta celda si decide descargar los archivos manualmente en el siguiente enlace:
https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/overview

Puede que necesite aceptar las reglas de la competencia en: https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/rules

Aceptelas y siga los pasos que se le muestren en pantalla.

In [4]:
if download_data:
    # Autenticación con Kaggle API
    kaggle.api.authenticate()

    try:
        kaggle.api.competition_download_files(competition=competition_name,
                                            path='data',
                                            quiet=False,
                                            force=False)
        
        print(f"EXITO: Archivos descargados en '{target_path}'")

    except ApiException as e:
        print(f"ERROR al llamar KaggleApi -> competition_download \n\t{e}")
else:
    print("Descarga de datos realizada previamente...")

Descarga de datos realizada previamente...


### Descomprimir el Zip

Puede comentar esta celda si ya tiene los archivos descomprimidos

In [5]:
if download_data:
    try:
        # Descomprimir archivo .zip
        with zipfile.ZipFile(f'{target_path}/{competition_name}.zip', 'r') as zip_ref:
            zip_ref.extractall(target_path)

        # Eliminar archivo .zip
        os.remove(f'{target_path}/{competition_name}.zip')

        print(f"EXITO: Archivo .zip descomprimido en '{target_path}'")

    except Exception as e:
        print(f"ERROR al descomprimir archivo .zip \n\t{e}")
else:
    print("Descompresión de archivos realizada previamente...")
    print(f"Conjunto de datos en '{target_path}'")

Descompresión de archivos realizada previamente...
Conjunto de datos en 'data'


Deberia de tener los siguientes archivos dentro de la carpeta `data`

- segmentations
  - archivos .nii
- test_images
  - archivos .dcm
- train_images
  - 1.2.826.0....
    - archivos .dcm
  - ...  
- sample_submission.csv
- test.csv
- train_bounding_boxes.csv
- train.csv

### Limpieza del entorno de trabajo

Puede comentar esta celda si no tiene estos archivos o si no desea eliminarlos.

Dado el enfoque académico de este análisis y la naturaleza de las competencias de Kaggle, los archivos relacionados con el conjunto de test no serán utilizados. Estos archivos tienen una estructura diferente al conjunto de entrenamiento y están destinados a evaluar los modelos generados en la competencia. Como no se pretende generar un modelo competitivo en este contexto, se procederá a eliminar los siguientes archivos/carpetas:
- test_images
- sampe_submission.csv
- test.csv

In [9]:
# Función para eliminar archivos o carpetas
def remove_files(paths):
    """
    Elimina archivos o carpetas de la lista de rutas especificadas.
    """
    for path in paths:
        if os.path.isdir(path):
            # Si es una carpeta, usar shutil.rmtree() para eliminarla
            try:
                shutil.rmtree(path)
                print(f"Carpeta eliminada: {path}")
            except Exception as e:
                print(f"Error al eliminar la carpeta {path}: {e}")

        elif os.path.isfile(path):
            # Si es un archivo, usar os.remove() para eliminarlo
            try:
                os.remove(path)
                print(f"Archivo eliminado: {path}")
            except Exception as e:
                print(f"Error al eliminar el archivo {path}: {e}")
        else:
            print(f"{path} no encontrado, omitiendo...")

In [10]:
# Definir los caminos a los archivos y carpetas a eliminar
paths_to_remove = [
    'data/test_images', 
    'data/sample_submission.csv', 
    'data/test.csv'
]

# Ejecutar la función
remove_files(paths_to_remove)

Carpeta eliminada: data/test_images
Archivo eliminado: data/sample_submission.csv
Archivo eliminado: data/test.csv
